In [1]:
import psycopg
import dotenv
import os
from openai import OpenAI


In [2]:
dotenv.load_dotenv()

client = OpenAI()

conn = psycopg.connect(
    f"dbname=postgres user=postgres password={os.environ.get('PG_PWD')} host=127.0.0.1 port=5432"
)

SEARCH_QUERY = "crypto"

In [3]:
def create_embedding(client, data):
    return client.embeddings.create(
        model="text-embedding-3-small",
        input=str(data),
        encoding_format="float",
    )

In [4]:
query_embedding = create_embedding(client, SEARCH_QUERY)


In [5]:
query_embedding_data = query_embedding.data[0].embedding


In [6]:
with conn.cursor() as cur:
    try:
        cur.execute(
            """
            SELECT id, title, body
            FROM documents
            ORDER BY embedding <=> %s::vector
            LIMIT 5;
            """,
            (query_embedding_data,),
        )
        results = cur.fetchall()
    finally:
        conn.rollback()
    for row in results:
        print(row)


(4, 'Crypto CEO Buys Volcano, Builds Thermal Mining Farm', 'Rumors swirled this week as leaked images revealed that Ivan "Magma" Tolensk, CEO of crypto firm LavaMint, has privately purchased a dormant volcano in El Salvador. Sources close to the matter suggest the crater will house geothermal crypto mining rigs. Skeptics cite local geological instability, while believers hail it as "the natural evolution of proof-of-work."')
(1, 'Sea Moss Currency? Barbados Launches Experimental Trade Token', 'In a bid to promote sustainable economic growth and capitalize on its marine resources, Barbados has announced the launch of the “MossCoin,” a blockchain-based token backed by sea moss harvests. Local cooperatives will receive MossCoin for each kilogram of certified organic sea moss, redeemable for agricultural tools, subsidies, and even inter-island ferry credits. Though skeptics have voiced concerns over its volatility, the government insists the initiative blends natural capital with modern fi